In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    DDD_Train_Decile4 = pd.read_pickle("DDD_Train_Decile4.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile4.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile4, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150,200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 4')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 4s 2ms/step - loss: 0.4550 - acc: 0.8140 - val_loss: 0.4373 - val_acc: 0.8148
Epoch 39/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4495 - acc: 0.8083 - val_loss: 0.4410 - val_acc: 0.8113
Epoch 40/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4495 - acc: 0.8149 - val_loss: 0.4435 - val_acc: 0.8042
Epoch 41/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4464 - acc: 0.8154 - val_loss: 0.4340 - val_acc: 0.8095
Epoch 42/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4475 - acc: 0.8048 - val_loss: 0.4356 - val_acc: 0.8148
Epoch 43/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4461 - acc: 0.8180 - val_loss: 0.4312 - val_acc: 0.8113
Epoch 44/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4418 - acc: 0.8114 - val_loss: 0.4262 - val_acc: 0.8201
Epoch 45/100
2264/2264 [==============================] - 4s 

Epoch 98/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4299 - acc: 0.8163 - val_loss: 0.4109 - val_acc: 0.8289
Epoch 99/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4306 - acc: 0.8123 - val_loss: 0.4107 - val_acc: 0.8289
Epoch 100/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4286 - acc: 0.8176 - val_loss: 0.4115 - val_acc: 0.8236
Test accuracy: 0.823633155915262
Train on 2264 samples, validate on 567 samples
Epoch 1/150
2264/2264 [==============================] - 6s 3ms/step - loss: 0.6468 - acc: 0.5839 - val_loss: 0.5957 - val_acc: 0.6402
Epoch 2/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.5519 - acc: 0.6974 - val_loss: 0.5183 - val_acc: 0.7037
Epoch 3/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.5292 - acc: 0.7593 - val_loss: 0.5024 - val_acc: 0.7619
Epoch 4/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.5006 - acc: 0.7871 - val_lo

2264/2264 [==============================] - 4s 2ms/step - loss: 0.3933 - acc: 0.8273 - val_loss: 0.3789 - val_acc: 0.8289
Epoch 57/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3929 - acc: 0.8282 - val_loss: 0.3788 - val_acc: 0.8289
Epoch 58/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3974 - acc: 0.8344 - val_loss: 0.3788 - val_acc: 0.8289
Epoch 59/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3985 - acc: 0.8260 - val_loss: 0.3786 - val_acc: 0.8307
Epoch 60/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3963 - acc: 0.8255 - val_loss: 0.3786 - val_acc: 0.8307
Epoch 61/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3922 - acc: 0.8269 - val_loss: 0.3787 - val_acc: 0.8307
Epoch 62/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3952 - acc: 0.8295 - val_loss: 0.3788 - val_acc: 0.8289
Epoch 63/150
2264/2264 [==============================] - 4s 

2264/2264 [==============================] - 4s 2ms/step - loss: 0.4001 - acc: 0.8282 - val_loss: 0.3782 - val_acc: 0.8307
Epoch 117/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4018 - acc: 0.8238 - val_loss: 0.3783 - val_acc: 0.8307
Epoch 118/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3997 - acc: 0.8313 - val_loss: 0.3784 - val_acc: 0.8289
Epoch 119/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3953 - acc: 0.8282 - val_loss: 0.3785 - val_acc: 0.8325
Epoch 120/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3963 - acc: 0.8251 - val_loss: 0.3786 - val_acc: 0.8325
Epoch 121/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3974 - acc: 0.8277 - val_loss: 0.3782 - val_acc: 0.8307
Epoch 122/150
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3988 - acc: 0.8269 - val_loss: 0.3782 - val_acc: 0.8307
Epoch 123/150
2264/2264 [==============================

...

2264/2264 [==============================] - 4s 2ms/step - loss: 0.4201 - acc: 0.8211 - val_loss: 0.4041 - val_acc: 0.8219
Epoch 67/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4236 - acc: 0.8083 - val_loss: 0.4065 - val_acc: 0.8236
Epoch 68/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4329 - acc: 0.8136 - val_loss: 0.4036 - val_acc: 0.8272
Epoch 69/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4223 - acc: 0.8180 - val_loss: 0.4019 - val_acc: 0.8325
Epoch 70/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4241 - acc: 0.8167 - val_loss: 0.4050 - val_acc: 0.8219
Epoch 71/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4243 - acc: 0.8167 - val_loss: 0.4018 - val_acc: 0.8307
Epoch 72/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4226 - acc: 0.8189 - val_loss: 0.4076 - val_acc: 0.8166
Epoch 73/100
2264/2264 [==============================] - 4s 

Epoch 25/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4002 - acc: 0.8286 - val_loss: 0.3856 - val_acc: 0.8325
Epoch 26/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4008 - acc: 0.8207 - val_loss: 0.3848 - val_acc: 0.8342
Epoch 27/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4043 - acc: 0.8220 - val_loss: 0.3850 - val_acc: 0.8325
Epoch 28/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4076 - acc: 0.8211 - val_loss: 0.3849 - val_acc: 0.8342
Epoch 29/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4120 - acc: 0.8264 - val_loss: 0.3847 - val_acc: 0.8325
Epoch 30/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4008 - acc: 0.8224 - val_loss: 0.3844 - val_acc: 0.8307
Epoch 31/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4069 - acc: 0.8229 - val_loss: 0.3851 - val_acc: 0.8289
Epoch 32/200
2264/2264 [========================

2264/2264 [==============================] - 4s 2ms/step - loss: 0.4009 - acc: 0.8238 - val_loss: 0.3836 - val_acc: 0.8325
Epoch 85/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4055 - acc: 0.8264 - val_loss: 0.3835 - val_acc: 0.8325
Epoch 86/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4010 - acc: 0.8282 - val_loss: 0.3835 - val_acc: 0.8342
Epoch 87/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3994 - acc: 0.8295 - val_loss: 0.3834 - val_acc: 0.8342
Epoch 88/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4037 - acc: 0.8242 - val_loss: 0.3835 - val_acc: 0.8342
Epoch 89/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3985 - acc: 0.8291 - val_loss: 0.3835 - val_acc: 0.8360
Epoch 90/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4075 - acc: 0.8286 - val_loss: 0.3835 - val_acc: 0.8360
Epoch 91/200
2264/2264 [==============================] - 4s 

2264/2264 [==============================] - 4s 2ms/step - loss: 0.4048 - acc: 0.8220 - val_loss: 0.3827 - val_acc: 0.8377
Epoch 145/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4045 - acc: 0.8326 - val_loss: 0.3827 - val_acc: 0.8377
Epoch 146/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.3966 - acc: 0.8238 - val_loss: 0.3828 - val_acc: 0.8360
Epoch 147/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4004 - acc: 0.8233 - val_loss: 0.3828 - val_acc: 0.8360
Epoch 148/200
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4035 - acc: 0.8277 - val_loss: 0.3828 - val_acc: 0.8360
Epoch 149/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4018 - acc: 0.8295 - val_loss: 0.3828 - val_acc: 0.8360
Epoch 150/200
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4032 - acc: 0.8308 - val_loss: 0.3828 - val_acc: 0.8360
Epoch 151/200
2264/2264 [==============================

2264/2264 [==============================] - 4s 2ms/step - loss: 0.5463 - acc: 0.7049 - val_loss: 0.5073 - val_acc: 0.7125
Epoch 4/50
2264/2264 [==============================] - 4s 2ms/step - loss: 0.5199 - acc: 0.7571 - val_loss: 0.4893 - val_acc: 0.7707
Epoch 5/50
2264/2264 [==============================] - 4s 2ms/step - loss: 0.5056 - acc: 0.7818 - val_loss: 0.4861 - val_acc: 0.7760
Epoch 6/50
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4949 - acc: 0.7968 - val_loss: 0.4853 - val_acc: 0.7989
Epoch 7/50
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4865 - acc: 0.8096 - val_loss: 0.4982 - val_acc: 0.7884
Epoch 8/50
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4807 - acc: 0.8043 - val_loss: 0.4635 - val_acc: 0.7954
Epoch 9/50
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4745 - acc: 0.8105 - val_loss: 0.4519 - val_acc: 0.7989
Epoch 10/50
2264/2264 [==============================] - 4s 2ms/step - lo

In [ ]:
0.86434